In [1]:
import pandas as pd
from time import sleep
import json

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
supplements = pd.read_csv("Supplements.csv")

In [4]:
supplements = supplements[supplements["Name"].str.startswith('A')]
supplements


,Name,Url
0,Abscess Root,https://www.webmd.com/vitamins/ai/ingredientmo...
1,Abuta,https://www.webmd.com/vitamins/ai/ingredientmo...
2,Acacia Rigidula,https://www.webmd.com/vitamins/ai/ingredientmo...
3,Acai,https://www.webmd.com/vitamins/ai/ingredientmo...
4,Acerola,https://www.webmd.com/vitamins/ai/ingredientmo...
...,...,...
100,Avens,https://www.webmd.com/vitamins/ai/ingredientmo...
101,Avocado,https://www.webmd.com/vitamins/ai/ingredientmo...
102,Avocado Soy Unsaponifiables (Asu),https://www.webmd.com/vitamins/ai/ingredientmo...
103,Avocado Sugar Extract,https://www.webmd.com/vitamins/ai/ingredientmo...


In [5]:
def get_overview(driver, supplement_name):
    overview = ""
    try:        
        target_div = driver.find_element(By.CSS_SELECTOR, '#overview-container .vitamins-monograph-content.overview-content')
        div_text = target_div.text

        overview = div_text

    except Exception:
        overview="Error"
    finally:
        return overview

In [6]:
supplement_info = {}

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait_time = WebDriverWait(driver,10)

for _, info in supplements.iterrows():

    supplementName = info.Name
    supplementURL = info.Url
    
    driver.get(supplementURL)       

    overview = get_overview(driver, supplementName)
    
    #overview_content = parent_monograph.find_elements(By.CLASS_NAME, "vitamins-monograph-content overview-content")
    #text = overview_content.text.strip()
    
    
    #overview = get_overview_content()
    #overview = get_element_text(supplementName, "Overview", "/html/body/div[1]/div/main/div/div[2]/div/div/div[1]/div[1]/div")
    #uses = get_uses("/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]")
    #sideEffects = get_element_text(supplementName, "Side Effects", "/html/body/div[1]/div/main/div/div[2]/div/div/div[3]/div[1]")
    #precautions = get_element_text(supplementName, "Precautions", "/html/body/div[1]/div/main/div/div[2]/div/div/div[4]/div[1]/div")
    #interactons = [] /html/body/div[1]/div/main/div/div[2]/div/div/div[4]/div[2]/div
    #dosing = get_element_text(supplementName, "Dosing", "/html/body/div[1]/div/main/div/div[2]/div/div/div[6]/div[1]/div")

    supplement_info[supplementName] = {
         "Overview": overview
    #     "Uses": [],
    #     "SideEffects": [],
    #     "Precautions": [],
    #     "Interactions": [],
    #     "Dosing": [] 
     }
    print(f"Done {supplementName}")
    sleep(3)
driver.quit()

Done Abscess Root
Done Abuta
Done Acacia Rigidula
Done Acai
Done Acerola
Done Acetyl-L-Carnitine
Done Ackee
Done Aconite
Done Activated Charcoal
Done Adenosine
Done Adrafinil
Done Adrenal Extract
Done Adrue
Done Aegeline
Done African Wild Potato
Done Agar
Done Agaricus Mushroom
Done Agave
Done Agmatine
Done Agrimony
Done Ajuga Nipponensis
Done Alanine
Done Albizia Julibrissin
Done Alchemilla
Done Alder Buckthorn
Done Aletris
Done Alfalfa
Done Algal Oil
Done Algin
Done Alkanna
Done Allspice
Done Aloe
Done Alpha-Gpc
Done Alpha-Ketoglutarate (Akg)
Done Alpha-Linolenic Acid (Ala)
Done Alpha-Lipoic Acid
Done Alpine Ragwort
Done Alpinia
Done Amaranth
Done Ambrette
Done American Adder'S Tongue
Done American Chestnut
Done American Dogwood
Done American Elder
Done American Ginseng
Done American Hellebore
Done American Ivy
Done American Mistletoe
Done American Pawpaw
Done American Spikenard
Done American White Water Lily
Done Andarine
Done Andiroba
Done Andrographis
Done Androstenediol
Done Andr

In [7]:
supplement_info

{'Abscess Root': {'Overview': 'Abscess root (Polemonium reptans) is an herb. The root is ground and used as a medicinal tea, but it\'s not clear if it has any benefits.\n\nAbscess root is used for fever, cough, swelling, and other conditions. But there is no good scientific evidence to support any uses.\n\nAbscess root is also called "false Jacob\'s ladder," because it has some of the same effects as another plant called Jacob\'s ladder. These two plants are used in similar ways, but are not the same.'},
 'Abuta': {'Overview': "Abuta (Cissampelos pareira) is a climbing plant that grows in tropical regions worldwide. Its bark, root, seed, and other parts are used as medicine.\n\nAbuta contains chemicals that may relax muscles and reduce pain.\n\nPeople use abuta for stomach pain, migraine, joint pain, wound healing, and many other conditions, but there is no good scientific evidence to support these uses.\n\nDon't confuse abuta with pareira. These are not the same."},
 'Acacia Rigidula'

In [136]:
print(json.dumps(supplement_info, indent=4))

{
    "Abscess Root": {
        "Overview": "Abscess root (Polemonium reptans) is an herb. The root is ground and used as a medicinal tea, but it's not clear if it has any benefits.\n\nAbscess root is used for fever, cough, swelling, and other conditions. But there is no good scientific evidence to support any uses.\n\nAbscess root is also called \"false Jacob's ladder,\" because it has some of the same effects as another plant called Jacob's ladder. These two plants are used in similar ways, but are not the same.",
        "Uses": [],
        "SideEffects": [],
        "Precautions": [],
        "Interactions": [],
        "Dosing": []
    }
}


TEST HERE

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.webmd.com/vitamins/ai/ingredientmono-266/abscess-root")
wait_time = WebDriverWait(driver,5)
#div = "/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]"
/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]/div[3]
div = "/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/p"
elem = wait_time.until(EC.presence_of_element_located((By.XPATH, div)))
elem = wait_time.until(EC.)
print(elem.text)
driver.quit()